In [1]:
#problem statement

In [2]:
#generate story similar to alice in wonderland

In [3]:
import torch
import os
import torch.nn as nn
import numpy as np
from torch.nn.utils import clip_grad_norm

In [4]:
class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0

    def add_word(self, word):
        if word not in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1
            
    def __len__(self):
        return len(self.word2idx)

In [11]:
import os
os.chdir('/content/drive/My Drive/NN-bootcamp')

In [12]:
%pwd

'/content/drive/My Drive/NN-bootcamp'

In [13]:
class TextProcess(object):
    
    def __init__(self):
        self.dictionary = Dictionary()

    def get_data(self, path, batch_size=20):
        with open(path, 'r') as f:
            tokens = 0
            for line in f:
                words = line.split() + ['<eos>']
                tokens += len(words)
                for word in words: 
                    self.dictionary.add_word(word)  
        #Create a 1-D tensor that contains the index of all the words in the file
        rep_tensor = torch.LongTensor(tokens)
        index = 0
        with open(path, 'r') as f:
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    rep_tensor[index] = self.dictionary.word2idx[word]
                    index += 1
        #Find out how many batches we need            
        num_batches = rep_tensor.shape[0] // batch_size     
        #Remove the remainder (Filter out the ones that don't fit)
        rep_tensor = rep_tensor[:num_batches*batch_size]
        # return (batch_size,num_batches)
        rep_tensor = rep_tensor.view(batch_size, -1)
        return rep_tensor

In [14]:
embed_size = 128    #Input features to the LSTM
hidden_size = 1024  #Number of LSTM units
num_layers = 1
num_epochs = 20
batch_size = 20
timesteps = 30
learning_rate = 0.002

In [16]:
!ls

basics-pytorch.ipynb  LSTM-Implementation.ipynb
cnn.ipynb	      The-Complete-Neural-Networks-Bootcamp-Theory-Applications
FFNN-Diabetes	      Visualize-NN


In [17]:
corpus=TextProcess()
rep_tensor = corpus.get_data('The-Complete-Neural-Networks-Bootcamp-Theory-Applications/alice.txt', batch_size)

In [23]:
vocab_size = len(corpus.dictionary)
print(vocab_size)

5290


In [24]:
num_batches = rep_tensor.shape[1] // timesteps
print(num_batches)

49


In [25]:
class TextGenerator(nn.Module):
    
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(TextGenerator, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, h):
        # Perform Word Embedding 
        x = self.embed(x)
        #Reshape the input tensor
        #x = x.view(batch_size,timesteps,embed_size)
        out, (h, c) = self.lstm(x, h)
        # Reshape the output from (samples,timesteps,output_features) to a shape appropriate for the FC layer 
        # (batch_size*timesteps, hidden_size)
        out = out.reshape(out.size(0)*out.size(1), out.size(2))
        # Decode hidden states of all time steps
        out = self.linear(out)
        return out, (h, c)

In [26]:
model = TextGenerator(vocab_size, embed_size, hidden_size, num_layers)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(num_epochs):
    # Set initial hidden and cell states
    states = (torch.zeros(num_layers, batch_size, hidden_size),
              torch.zeros(num_layers, batch_size, hidden_size))

    for i in range(0, rep_tensor.size(1) - timesteps, timesteps):
        # Get mini-batch inputs and targets
        inputs = rep_tensor[:, i:i+timesteps]  
        targets = rep_tensor[:, (i+1):(i+1)+timesteps]
        
        outputs,_ = model(inputs, states)
        loss = loss_fn(outputs, targets.reshape(-1))

        model.zero_grad()
        loss.backward()
        #Perform Gradient Clipping. clip_value (float or int) is the maximum allowed value of the gradients 
        #The gradients are clipped in the range [-clip_value, clip_value]. This is to prevent the exploding gradient problem
        clip_grad_norm(model.parameters(), 0.5)
        optimizer.step()
              
        step = (i+1) // timesteps
        if step % 100 == 0:
            print ('Epoch [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, loss.item()))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


Epoch [1/20], Loss: 8.5765
Epoch [2/20], Loss: 5.9709
Epoch [3/20], Loss: 5.2460
Epoch [4/20], Loss: 4.7196
Epoch [5/20], Loss: 4.1653
Epoch [6/20], Loss: 3.6985
Epoch [7/20], Loss: 3.1947
Epoch [8/20], Loss: 2.7880
Epoch [9/20], Loss: 2.5318
Epoch [10/20], Loss: 2.1924
Epoch [11/20], Loss: 1.8997
Epoch [12/20], Loss: 1.6276
Epoch [13/20], Loss: 1.3281
Epoch [14/20], Loss: 1.0678
Epoch [15/20], Loss: 0.8262
Epoch [16/20], Loss: 0.5356
Epoch [17/20], Loss: 0.4591
Epoch [18/20], Loss: 0.2545
Epoch [19/20], Loss: 0.1457
Epoch [20/20], Loss: 0.0977
